In [16]:
import os
import rasterio
import numpy as np
from rasterio.warp import transform_bounds


def get_bbox_intersect_2_img(list_fp_img):
    """ Tim giao nhau giua 2 anh va tra ve bbox cua vung giao nhau

    Args:
        list_fp_img (list): gom 2 cai anh tif, nen la cung toa do nhe

    Returns:
        _type_: tuple, bbox cua giao nhau
    """
    list_bound = []
    list_crs = []
    for fp_img in list_fp_img:
        with rasterio.open(fp_img) as src:
            bounds = src.bounds
            crs = src.crs
        list_bound.append(bounds)
        list_crs.append(crs)
        
     
    bound_left = [bounds.left for bounds in list_bound]
    bound_bottom = [bounds.bottom for bounds in list_bound]
    bound_right = [bounds.right for bounds in list_bound]
    bound_top = [bounds.top for bounds in list_bound]

    xmin = max(bound_left)
    ymin = max(bound_bottom)
    xmax = min(bound_right) 
    ymax = min(bound_top)

    leftw, bottomw, rightw, topw = transform_bounds(list_crs[0], list_crs[1], xmin, ymin, xmax, ymax)
    left, bottom, right, top = transform_bounds(list_crs[1], list_crs[0], leftw, bottomw, rightw, topw)
    return (left, bottom, right, top)



def clip_raster_by_bbox(input_path, bbox, output_path= None, return_ = True, export_file_tiff = True):
    with rasterio.open(input_path) as src:
        # Get the window to read from
        minx, miny, maxx, maxy = bbox
        window = src.window(minx, miny, maxx, maxy)

        # Calculate the width and height of the output
        width = window.width
        height = window.height

        # Compute the transform for the output
        transform = rasterio.windows.transform(window, src.transform)
        nodata = src.nodata
        # Update the metadata for the output
        meta = src.meta.copy()
        meta.update({
            'driver': 'GTiff',
            'height': height,
            'width': width,
            'transform': transform
        })
        if export_file_tiff:
        # Read and write the data
            with rasterio.open(output_path, 'w', **meta) as dst:
                dst.write(src.read(window=window))
        if return_:
            return src.read(window=window), meta, nodata
 


fp_in_truoc = r'/home/skm/SKM16/Tmp/XONG_XOAAAAAAAAAAAAAAAAAAAAAAAA/Img/S1A_IW_GRDH_1SDV_20220323T215024_0.tif'
fp_in_sau = r'/home/skm/SKM16/Tmp/XONG_XOAAAAAAAAAAAAAAAAAAAAAAAA/Img/S1A_IW_GRDH_1SDV_20220416T215025_0.tif'
fp_out = r'/home/skm/SKM16/Tmp/XONG_XOAAAAAAAAAAAAAAAAAAAAAAAA/Img/rs/change20220323_vs_20220416_v2100.tif'
os.makedirs(os.path.dirname(fp_out), exist_ok=True)

bbox = get_bbox_intersect_2_img([fp_in_truoc, fp_in_sau])
img_truoc, meta, nodata1 = clip_raster_by_bbox(fp_in_truoc, bbox, return_ = True, export_file_tiff = False)
img_sau, meta, nodata2 = clip_raster_by_bbox(fp_in_sau, bbox, return_ = True, export_file_tiff = False)

meta.update({'count':1})
ind_nodata_truoc = np.where(img_truoc==nodata1)
ind_nodata_sau = np.where(img_sau==nodata2)
change = np.empty_like(img_sau)
change = img_sau - img_truoc
change[ind_nodata_truoc] = nodata1
change[ind_nodata_sau] = nodata2

with rasterio.open(fp_out, 'w', **meta) as dst:
    dst.write(change)
